# Reto: Análisis del progreso mundial de vacunación

Libreta en **Google Colab** usando **pandas** y **numpy**
- Jorge Adán Avila Gonzalez

**Objetivo general**: recuperar, seleccionar, procesar y sintetizar información del archivo `country_vaccinations.csv` para obtener resultados concluyentes sobre el **avance de vacunación** por país.

**Archivos esperados**:
- `country_vaccinations.csv` (cárgalo en la misma carpeta de la libreta, o subir con el cuadro de carga).
- La libreta generará un archivo **`resultadosReto.xlsx`**


## (a) Extraer la información del archivo [Carga del archivo CSV]
Intenta leer el CSV desde rutas comunes en Colab. Si no se encuentra, se abrirá un cuadro para **cargar** el archivo manualmente.

In [15]:
import os
import numpy as np
import pandas as pd

def cargar_csv_posibles(rutas):
    for ruta in rutas:
        if os.path.exists(ruta):
            try:
                return pd.read_csv(ruta)
            except Exception as e:
                print(f"Intento fallido con {ruta}: {e}")
    return None

posibles_rutas = [
    '/content/country_vaccinations.csv',   # ruta común en Colab
    './country_vaccinations.csv',         # carpeta actual
    '/mnt/data/country_vaccinations.csv'  # ruta opcional (por si ya existe en este entorno)
]

df = cargar_csv_posibles(posibles_rutas)

if df is None:
    # Fallback para subir manualmente en Colab
    try:
        from google.colab import files  # solo disponible en Colab
        print("Sube el archivo country_vaccinations.csv")
        up = files.upload()
        nombre = list(up.keys())[0]
        df = pd.read_csv(nombre)
    except Exception as e:
        raise RuntimeError("No se pudo cargar el archivo automáticamente y no está en Colab para subirlo.") from e

print('Filas:', len(df), '| Columnas:', df.shape[1])
df.head(3) #(TOP 3 para facilitar la lectura)

Filas: 86512 | Columnas: 15


,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/


## (b) Estructura y tipos de datos de cada columna
- Conversión de columnas de **fecha** a `datetime64`.
- Revisión de **valores nulos** y de **tipos**.

In [41]:
# Copia de trabajo para no modificar el original accidentalmente (backup)
data = df.copy()

# Convertir columna de fecha a datetime (si se llama 'date')
if 'date' in data.columns:
    data['date'] = pd.to_datetime(data['date'], errors='coerce')

# Mostrar estructura general
print('Dimensiones:', data.shape)
print('\nTipos de dato:')
print(data.dtypes)

print('\nValores nulos por columna:')
print(data.isna().sum().apply(lambda x: f"{x:,.0f}")) #formateo las salidas numericas (,.)

Dimensiones: (86512, 15)

Tipos de dato:
country                                        object
iso_code                                       object
date                                   datetime64[ns]
total_vaccinations                            float64
people_vaccinated                             float64
people_fully_vaccinated                       float64
daily_vaccinations_raw                        float64
daily_vaccinations                            float64
total_vaccinations_per_hundred                float64
people_vaccinated_per_hundred                 float64
people_fully_vaccinated_per_hundred           float64
daily_vaccinations_per_million                float64
vaccines                                       object
source_name                                    object
source_website                                 object
dtype: object

Valores nulos por columna:
country                                     0
iso_code                                    0
date           

## (c) Cantidad de vacunas **aplicadas por compañía**
Para cumplir la rúbrica, contamos por **compañía individual**. Partimos de la columna `vaccines`, la separamos por coma, expandimos filas y **sumamos** las inoculaciones diarias.

In [43]:
# Conteo por compañía individual usando daily vaccinations
tmp = data[[COL_DATE, COL_COUNTRY, COL_DAILY, COL_VACCINES]].dropna(subset=[COL_DAILY, COL_VACCINES]).copy()
tmp['compania'] = tmp[COL_VACCINES].str.split(',')
tmp = tmp.explode('compania')
tmp['compania'] = tmp['compania'].str.strip()

vacunas_por_compania = (
    tmp.groupby('compania', as_index=False)[COL_DAILY]
       .sum()
       .sort_values(COL_DAILY, ascending=False)
)

# ---- Formato con comas de miles ----
vacunas_por_compania['vacunas_formateadas'] = vacunas_por_compania[COL_DAILY].apply(lambda x: f"{x:,.0f}")

# Mostrar solo las columnas relevantes con formato legible (TOP 10 para facilitar la lectura)
display(vacunas_por_compania[['compania', 'vacunas_formateadas']].head(10))

,compania,vacunas_formateadas
10,Oxford/AstraZeneca,"6,952,965,433"
16,Sinovac,"5,809,994,990"
14,Sinopharm/Beijing,"5,687,869,560"
11,Pfizer/BioNTech,"5,547,633,770"
8,Moderna,"4,291,922,376"
2,CanSino,"3,906,599,041"
21,Sputnik V,"3,586,652,576"
6,Johnson&Johnson,"3,567,280,935"
23,ZF2001,"3,301,927,868"
15,Sinopharm/Wuhan,"3,274,047,095"


## (d) Cantidad de vacunas aplicadas en **todo el mundo**

In [45]:
total_mundial = data[COL_DAILY].sum(min_count=1)
print(f"{total_mundial:,.0f}")
#total_mundial

11,320,239,871


## (e) Promedio de vacunas aplicadas **por país**

In [46]:
promedio_por_pais = (
    data.groupby(COL_COUNTRY, as_index=False)[COL_DAILY]
        .mean(numeric_only=True)
        .rename(columns={COL_DAILY: 'promedio_diario'})
).sort_values('promedio_diario', ascending=False)

# Formatear columna con comas de miles
promedio_por_pais['promedio_formateado'] = promedio_por_pais['promedio_diario'].apply(lambda x: f"{x:,.0f}")

# Mostrar solo columnas relevantes (formateadas) (TOP 10 para facilitar la lectura)
display(promedio_por_pais[['country', 'promedio_formateado']].head(10))

,country,promedio_formateado
41,China,"6,930,368"
91,India,"4,175,994"
212,United States,"1,191,727"
27,Brazil,"943,529"
92,Indonesia,"846,289"
100,Japan,"621,580"
15,Bangladesh,"545,305"
151,Pakistan,"543,005"
217,Vietnam,"531,095"
127,Mexico,"413,425"


## (f) Cantidad de vacunas aplicadas el **29/01/2021** (mundo)

In [47]:
fecha_objetivo = pd.Timestamp('2021-01-29')
en_fecha = data.loc[data[COL_DATE] == fecha_objetivo, COL_DAILY]
total_290121 = en_fecha.sum(min_count=1)
print(f"{total_290121:,.0f}")
#total_290121

4,884,052


## (g) DataFrame **conDiferencias** (columna `diferencias` = daily − daily_raw)

In [48]:
conDiferencias = data.copy()
# Convertir a numérico (por si hay strings) y manejar nulos como 0 para la diferencia
conDiferencias[COL_DAILY] = pd.to_numeric(conDiferencias[COL_DAILY], errors='coerce')
conDiferencias[COL_DAILY_RAW] = pd.to_numeric(conDiferencias[COL_DAILY_RAW], errors='coerce')
conDiferencias['diferencias'] = (
    conDiferencias[COL_DAILY].fillna(0) - conDiferencias[COL_DAILY_RAW].fillna(0)
)
conDiferencias[['country','date',COL_DAILY, COL_DAILY_RAW, 'diferencias']].head(10) #(TOP 10 para facilitar la lectura)

,country,date,daily_vaccinations,daily_vaccinations_raw,diferencias
0,Afghanistan,2021-02-22,NaN,NaN,0.0
1,Afghanistan,2021-02-23,1367.0,NaN,1367.0
2,Afghanistan,2021-02-24,1367.0,NaN,1367.0
3,Afghanistan,2021-02-25,1367.0,NaN,1367.0
4,Afghanistan,2021-02-26,1367.0,NaN,1367.0
5,Afghanistan,2021-02-27,1367.0,NaN,1367.0
6,Afghanistan,2021-02-28,1367.0,NaN,1367.0
7,Afghanistan,2021-03-01,1580.0,NaN,1580.0
8,Afghanistan,2021-03-02,1794.0,NaN,1794.0
9,Afghanistan,2021-03-03,2008.0,NaN,2008.0


## (h) Diferencia de **tiempo** entre la fecha más reciente y la más antigua


In [9]:
fecha_min = data[COL_DATE].min()
fecha_max = data[COL_DATE].max()
periodo = fecha_max - fecha_min
print('Mínima:', fecha_min, '| Máxima:', fecha_max, '| Diferencia:', periodo)
periodo

Mínima: 2020-12-02 00:00:00 | Máxima: 2022-03-29 00:00:00 | Diferencia: 482 days 00:00:00


Timedelta('482 days 00:00:00')

## (i) DataFrame **conCantidad**
Nueva columna `canVac` con la **cantidad de tipos de vacuna** listados por fila (separados por coma).

In [49]:
conCantidad = data.copy()
conCantidad['canVac'] = (
    conCantidad[COL_VACCINES]
      .fillna('')
      .apply(lambda s: 0 if s.strip()=='' else len([v.strip() for v in s.split(',') if v.strip()!='']))
)
conCantidad[[COL_COUNTRY, COL_DATE, COL_VACCINES, 'canVac']].head(10) #(TOP 10 para facilitar la lectura)

,country,date,vaccines,canVac
0,Afghanistan,2021-02-22,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",4
1,Afghanistan,2021-02-23,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",4
2,Afghanistan,2021-02-24,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",4
3,Afghanistan,2021-02-25,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",4
4,Afghanistan,2021-02-26,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",4
5,Afghanistan,2021-02-27,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",4
6,Afghanistan,2021-02-28,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",4
7,Afghanistan,2021-03-01,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",4
8,Afghanistan,2021-03-02,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",4
9,Afghanistan,2021-03-03,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",4


## (j) DataFrame **antes20** (fechas **antes del 20/12/2020**)

In [50]:
corte = pd.Timestamp('2020-12-20')
antes20 = data[data[COL_DATE] < corte].copy()
len(antes20), antes20.head(3) #(TOP 3 para facilitar la lectura)

(70,
       country iso_code       date  total_vaccinations  people_vaccinated  \
 13403  Canada      CAN 2020-12-14                 5.0                5.0   
 13404  Canada      CAN 2020-12-15               727.0              727.0   
 13405  Canada      CAN 2020-12-16              3025.0             3025.0   
 
        people_fully_vaccinated  daily_vaccinations_raw  daily_vaccinations  \
 13403                      NaN                     NaN                 NaN   
 13404                      NaN                   722.0               722.0   
 13405                      NaN                  2298.0              1510.0   
 
        total_vaccinations_per_hundred  people_vaccinated_per_hundred  \
 13403                            0.00                           0.00   
 13404                            0.00                           0.00   
 13405                            0.01                           0.01   
 
        people_fully_vaccinated_per_hundred  daily_vaccinations_per_milli

## (k) DataFrame **pfizer** (registros que usaron Pfizer)

In [51]:
pfizer = data[data[COL_VACCINES].str.contains('pfizer', case=False, na=False)].copy()
len(pfizer), pfizer.head(3) #(TOP 3 para facilitar la lectura)

(64193,
        country iso_code       date  total_vaccinations  people_vaccinated  \
 0  Afghanistan      AFG 2021-02-22                 0.0                0.0   
 1  Afghanistan      AFG 2021-02-23                 NaN                NaN   
 2  Afghanistan      AFG 2021-02-24                 NaN                NaN   
 
    people_fully_vaccinated  daily_vaccinations_raw  daily_vaccinations  \
 0                      NaN                     NaN                 NaN   
 1                      NaN                     NaN              1367.0   
 2                      NaN                     NaN              1367.0   
 
    total_vaccinations_per_hundred  people_vaccinated_per_hundred  \
 0                             0.0                            0.0   
 1                             NaN                            NaN   
 2                             NaN                            NaN   
 
    people_fully_vaccinated_per_hundred  daily_vaccinations_per_million  \
 0                     

## (l) Guardar **todos los DataFrames** en `resultadosReto.xlsx`
Cada DataFrame quedará en una hoja distinta con el nombre indicado.

In [52]:
salida_excel = 'resultadosReto.xlsx' #Archivo de salida

# Función auxiliar para formatear números con comas
def formatear_numeros(df):
    df_formateado = df.copy()
    for col in df_formateado.select_dtypes(include=[np.number]).columns:
        df_formateado[col] = df_formateado[col].apply(lambda x: f"{x:,.0f}")
    return df_formateado

# Aplicar formato a todos los DataFrames antes de exportar
vacunas_fmt = formatear_numeros(vacunas_por_compania)
promedios_fmt = formatear_numeros(promedio_por_pais)
dif_fmt = formatear_numeros(conDiferencias)
cant_fmt = formatear_numeros(conCantidad)
antes_fmt = formatear_numeros(antes20)
pfizer_fmt = formatear_numeros(pfizer)

# total_mundial como DataFrame formateado
total_fmt = pd.DataFrame({'total_mundial': [f"{total_mundial:,.0f}"]})

# Guardar en Excel con datos formateados
with pd.ExcelWriter(salida_excel, engine='openpyxl') as writer:
    vacunas_fmt.to_excel(writer, sheet_name='vacunas_por_compania', index=False)
    total_fmt.to_excel(writer, sheet_name='total_mundial', index=False)
    promedios_fmt.to_excel(writer, sheet_name='promedio_por_pais', index=False)
    dif_fmt.to_excel(writer, sheet_name='conDiferencias', index=False)
    cant_fmt.to_excel(writer, sheet_name='conCantidad', index=False)
    antes_fmt.to_excel(writer, sheet_name='antes20', index=False)
    pfizer_fmt.to_excel(writer, sheet_name='pfizer', index=False)

print(f"Archivo Excel generado correctamente: {salida_excel}")

Archivo Excel generado correctamente: resultadosReto.xlsx


### Descargar el archivo de resultados en Colab
Ejecuta esta celda **solo** si estás en Google Colab para descargar `resultadosReto.xlsx` al PC local.

In [53]:
try:
    from google.colab import files
    files.download('resultadosReto.xlsx')
except Exception:
    pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>